# LDOS Data Exploration

This notebook provides interactive exploration of experiment results from the LDOS tracing harness.

## Contents
1. Setup and Data Loading
2. Data Overview
3. Success Rate Analysis
4. Latency Distribution Visualization
5. Outlier Detection
6. Scenario Comparison
7. Export Filtered Data

## 1. Setup and Data Loading

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore')

# Project paths
WS_ROOT = Path('..').resolve()
RESULTS_DIR = WS_ROOT / 'results'
ANALYSIS_OUTPUT = WS_ROOT / 'analysis' / 'output'

print(f"Workspace: {WS_ROOT}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Available result directories: {[d.name for d in RESULTS_DIR.iterdir() if d.is_dir()] if RESULTS_DIR.exists() else 'None'}")

In [ ]:
def load_combined_results(results_dir: Path) -> pd.DataFrame:
    """
    Load all experiment results from the results directory.
    Combines results from multiple scenarios into a single DataFrame.
    """
    all_rows = []
    
    if not results_dir.exists():
        print(f"Results directory not found: {results_dir}")
        return pd.DataFrame()
    
    for scenario_dir in results_dir.iterdir():
        if not scenario_dir.is_dir():
            continue
            
        scenario_name = scenario_dir.name
        
        # Load JSON result files
        json_files = list(scenario_dir.glob('*_result.json'))
        
        for json_file in json_files:
            try:
                with open(json_file) as f:
                    data = json.load(f)
                data['scenario'] = scenario_name
                data['source_file'] = json_file.name
                all_rows.append(data)
            except Exception as e:
                print(f"Warning: Failed to load {json_file}: {e}")
        
        # Also try loading summary CSV if exists
        summary_csv = scenario_dir / 'summary.csv'
        if summary_csv.exists():
            try:
                df_summary = pd.read_csv(summary_csv)
                df_summary['scenario'] = scenario_name
                df_summary['source_file'] = 'summary.csv'
                # Append rows that aren't already loaded
                if len(json_files) == 0:
                    all_rows.extend(df_summary.to_dict('records'))
            except Exception as e:
                print(f"Warning: Failed to load {summary_csv}: {e}")
    
    if not all_rows:
        print("No data found. Run experiments first with: make run_all")
        return pd.DataFrame()
    
    df = pd.DataFrame(all_rows)
    print(f"Loaded {len(df)} trials from {df['scenario'].nunique()} scenarios")
    return df

# Load data
df = load_combined_results(RESULTS_DIR)

## 2. Data Overview

In [ ]:
# Basic info
if len(df) > 0:
    print("=== DataFrame Info ===")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nScenarios: {df['scenario'].unique().tolist()}")
    print(f"\nData types:")
    print(df.dtypes)
else:
    print("No data loaded. Please run experiments first.")

In [ ]:
# Statistical summary
if len(df) > 0:
    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    latency_cols = [c for c in numeric_cols if 'latency' in c.lower() or 'ms' in c.lower()]
    
    print("=== Latency Columns ===")
    print(latency_cols)
    
    if latency_cols:
        print("\n=== Latency Statistics ===")
        display(df[latency_cols].describe())

In [ ]:
# Sample data
if len(df) > 0:
    print("=== Sample Rows ===")
    display(df.head(10))

## 3. Success Rate Analysis

In [ ]:
if len(df) > 0 and 'status' in df.columns:
    # Success rate per scenario
    success_rates = df.groupby('scenario').apply(
        lambda x: (x['status'] == 'success').mean() * 100
    ).reset_index(name='success_rate_pct')
    
    print("=== Success Rate by Scenario ===")
    display(success_rates)
    
    # Visualize
    fig, ax = plt.subplots(figsize=(10, 5))
    bars = ax.bar(success_rates['scenario'], success_rates['success_rate_pct'], 
                  color=sns.color_palette('husl', len(success_rates)))
    ax.axhline(y=90, color='r', linestyle='--', label='90% threshold')
    ax.set_xlabel('Scenario')
    ax.set_ylabel('Success Rate (%)')
    ax.set_title('Success Rate by Scenario')
    ax.set_ylim(0, 105)
    ax.legend()
    
    # Add value labels
    for bar, rate in zip(bars, success_rates['success_rate_pct']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{rate:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
elif len(df) > 0:
    print("No 'status' column found. All trials assumed successful.")

In [ ]:
# Trial counts
if len(df) > 0:
    trial_counts = df.groupby('scenario').size().reset_index(name='n_trials')
    print("=== Trial Counts ===")
    display(trial_counts)

## 4. Latency Distribution Visualization

In [ ]:
def plot_latency_distributions(df: pd.DataFrame, latency_col: str = 'total_latency_ms'):
    """Plot latency distributions across scenarios."""
    if latency_col not in df.columns:
        print(f"Column '{latency_col}' not found. Available: {df.columns.tolist()}")
        return
    
    # Filter successful trials if status exists
    if 'status' in df.columns:
        df_plot = df[df['status'] == 'success'].copy()
    else:
        df_plot = df.copy()
    
    scenarios = df_plot['scenario'].unique()
    n_scenarios = len(scenarios)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Histogram per scenario
    ax1 = axes[0, 0]
    for scenario in scenarios:
        data = df_plot[df_plot['scenario'] == scenario][latency_col].dropna()
        ax1.hist(data, bins=30, alpha=0.6, label=scenario, density=True)
    ax1.set_xlabel(f'{latency_col}')
    ax1.set_ylabel('Density')
    ax1.set_title('Latency Distribution by Scenario')
    ax1.legend()
    
    # 2. Box plot
    ax2 = axes[0, 1]
    df_plot.boxplot(column=latency_col, by='scenario', ax=ax2)
    ax2.set_xlabel('Scenario')
    ax2.set_ylabel(f'{latency_col}')
    ax2.set_title('Latency Box Plot')
    plt.suptitle('')  # Remove auto-title
    
    # 3. Violin plot
    ax3 = axes[1, 0]
    sns.violinplot(data=df_plot, x='scenario', y=latency_col, ax=ax3)
    ax3.set_xlabel('Scenario')
    ax3.set_ylabel(f'{latency_col}')
    ax3.set_title('Latency Violin Plot')
    
    # 4. CDF
    ax4 = axes[1, 1]
    for scenario in scenarios:
        data = df_plot[df_plot['scenario'] == scenario][latency_col].dropna().sort_values()
        cdf = np.arange(1, len(data) + 1) / len(data)
        ax4.plot(data, cdf, label=scenario, linewidth=2)
    ax4.set_xlabel(f'{latency_col}')
    ax4.set_ylabel('CDF')
    ax4.set_title('Cumulative Distribution Function')
    ax4.legend()
    ax4.axhline(y=0.95, color='r', linestyle='--', alpha=0.5, label='P95')
    
    plt.tight_layout()
    plt.show()

if len(df) > 0:
    # Try common latency column names
    for col in ['total_latency_ms', 'planning_latency_ms', 'execution_latency_ms', 'latency_ms']:
        if col in df.columns:
            plot_latency_distributions(df, col)
            break
    else:
        print("No latency columns found.")

In [ ]:
# Compare multiple latency metrics
if len(df) > 0:
    latency_cols = [c for c in df.columns if 'latency' in c.lower()]
    
    if len(latency_cols) > 1:
        # Filter successful
        if 'status' in df.columns:
            df_success = df[df['status'] == 'success']
        else:
            df_success = df
        
        # Melt for plotting
        df_melted = df_success.melt(
            id_vars=['scenario'],
            value_vars=latency_cols,
            var_name='metric',
            value_name='latency_ms'
        )
        
        fig, ax = plt.subplots(figsize=(14, 6))
        sns.boxplot(data=df_melted, x='scenario', y='latency_ms', hue='metric', ax=ax)
        ax.set_xlabel('Scenario')
        ax.set_ylabel('Latency (ms)')
        ax.set_title('Latency Breakdown by Scenario')
        ax.legend(title='Metric', bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.tight_layout()
        plt.show()

## 5. Outlier Detection

In [ ]:
def detect_outliers_iqr(df: pd.DataFrame, column: str, k: float = 1.5) -> pd.DataFrame:
    """
    Detect outliers using the IQR method.
    
    Args:
        df: DataFrame
        column: Column to check for outliers
        k: IQR multiplier (default 1.5 for standard outliers, 3.0 for extreme)
    
    Returns:
        DataFrame with outlier rows
    """
    if column not in df.columns:
        return pd.DataFrame()
    
    data = df[column].dropna()
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    print(f"=== Outlier Detection for {column} ===")
    print(f"Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
    print(f"Outliers: {len(outliers)} / {len(df)} ({len(outliers)/len(df)*100:.1f}%)")
    
    return outliers

if len(df) > 0:
    for col in ['total_latency_ms', 'planning_latency_ms', 'execution_latency_ms']:
        if col in df.columns:
            outliers = detect_outliers_iqr(df, col)
            if len(outliers) > 0:
                print(f"\nOutlier scenarios: {outliers['scenario'].value_counts().to_dict()}")
            print()

In [ ]:
# Visualize outliers
if len(df) > 0 and 'total_latency_ms' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    col = 'total_latency_ms'
    
    # Box plot showing outliers
    ax1 = axes[0]
    df.boxplot(column=col, by='scenario', ax=ax1, showfliers=True)
    ax1.set_title('With Outliers')
    ax1.set_xlabel('Scenario')
    ax1.set_ylabel(col)
    plt.suptitle('')
    
    # Without extreme outliers
    ax2 = axes[1]
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df_no_extreme = df[(df[col] >= Q1 - 3*IQR) & (df[col] <= Q3 + 3*IQR)]
    df_no_extreme.boxplot(column=col, by='scenario', ax=ax2, showfliers=True)
    ax2.set_title('Without Extreme Outliers (3*IQR)')
    ax2.set_xlabel('Scenario')
    ax2.set_ylabel(col)
    plt.suptitle('')
    
    plt.tight_layout()
    plt.show()

## 6. Scenario Comparison

In [ ]:
def compare_scenarios_summary(df: pd.DataFrame, latency_col: str = 'total_latency_ms') -> pd.DataFrame:
    """
    Generate summary statistics per scenario.
    """
    if latency_col not in df.columns:
        return pd.DataFrame()
    
    # Filter successful
    if 'status' in df.columns:
        df_success = df[df['status'] == 'success']
    else:
        df_success = df
    
    summary = df_success.groupby('scenario')[latency_col].agg([
        ('n_trials', 'count'),
        ('mean', 'mean'),
        ('std', 'std'),
        ('min', 'min'),
        ('p25', lambda x: x.quantile(0.25)),
        ('median', 'median'),
        ('p75', lambda x: x.quantile(0.75)),
        ('p95', lambda x: x.quantile(0.95)),
        ('p99', lambda x: x.quantile(0.99)),
        ('max', 'max'),
    ]).round(2)
    
    return summary

if len(df) > 0:
    for col in ['total_latency_ms', 'planning_latency_ms', 'execution_latency_ms']:
        if col in df.columns:
            print(f"\n=== {col} Summary ===")
            summary = compare_scenarios_summary(df, col)
            display(summary)

In [ ]:
# Percentage change from baseline
if len(df) > 0 and 'scenario' in df.columns and 'total_latency_ms' in df.columns:
    col = 'total_latency_ms'
    
    # Filter successful
    if 'status' in df.columns:
        df_success = df[df['status'] == 'success']
    else:
        df_success = df
    
    means = df_success.groupby('scenario')[col].mean()
    
    if 'baseline' in means.index:
        baseline_mean = means['baseline']
        pct_change = ((means - baseline_mean) / baseline_mean * 100).round(1)
        
        print("=== Percentage Change from Baseline ===")
        for scenario, change in pct_change.items():
            if scenario != 'baseline':
                print(f"  {scenario}: {change:+.1f}%")

## 7. Export Filtered Data

In [ ]:
def export_filtered_data(df: pd.DataFrame, output_dir: Path, prefix: str = 'filtered'):
    """
    Export filtered datasets for further analysis.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Full dataset
    full_path = output_dir / f'{prefix}_all.csv'
    df.to_csv(full_path, index=False)
    print(f"Exported all data: {full_path}")
    
    # Successful trials only
    if 'status' in df.columns:
        df_success = df[df['status'] == 'success']
        success_path = output_dir / f'{prefix}_success_only.csv'
        df_success.to_csv(success_path, index=False)
        print(f"Exported successful trials: {success_path}")
    
    # Per-scenario exports
    for scenario in df['scenario'].unique():
        scenario_df = df[df['scenario'] == scenario]
        scenario_path = output_dir / f'{prefix}_{scenario}.csv'
        scenario_df.to_csv(scenario_path, index=False)
        print(f"Exported {scenario}: {scenario_path}")

# Uncomment to export
# if len(df) > 0:
#     export_filtered_data(df, ANALYSIS_OUTPUT / 'filtered_data')

In [ ]:
# Export summary table for LaTeX
def export_latex_table(df: pd.DataFrame, output_path: Path, latency_col: str = 'total_latency_ms'):
    """
    Export summary as LaTeX table.
    """
    summary = compare_scenarios_summary(df, latency_col)
    
    # Format for LaTeX
    latex = summary[['n_trials', 'mean', 'std', 'p95', 'max']].to_latex(
        float_format='%.2f',
        caption=f'Latency Summary ({latency_col})',
        label='tab:latency_summary'
    )
    
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w') as f:
        f.write(latex)
    
    print(f"Exported LaTeX table: {output_path}")
    print("\n" + latex)

# Uncomment to export
# if len(df) > 0:
#     export_latex_table(df, ANALYSIS_OUTPUT / 'tables' / 'latency_summary.tex')

## Next Steps

After exploring the data, proceed to:

1. **`02_statistical_analysis.ipynb`** - Statistical tests, confidence intervals, effect sizes
2. **`03_path_analysis.ipynb`** - End-to-end path analysis and callback chain visualization

Run experiments with:
```bash
cd ~/ldos_manip_tracing
make run_all NUM_TRIALS=30
```